In [12]:
import os
import glob
import pandas as pd
import shutil
import random
from ultralytics import YOLO
import mlflow
from pathlib import Path

In [3]:
BASE_DIR = "/home/meth/code/mlopsfin"

In [5]:
tt = pd.read_csv(os.path.join(BASE_DIR, "data_pipeline/flow/labels.csv"))
tt

,ImageID,XMin,XMax,YMin,YMax,LabelName_Text
0,00253a1cfb65edf6,0.000000,0.291667,0.837500,0.999375,Car
1,00253a1cfb65edf6,0.759167,0.999167,0.776250,0.998125,Car
2,05014ec6d0c7bd13,0.010006,0.083802,0.251390,0.358176,Car
3,05014ec6d0c7bd13,0.055660,0.268293,0.208009,0.567297,Car
4,05014ec6d0c7bd13,0.167605,0.783615,0.052280,0.998888,Car
...,...,...,...,...,...,...
421,f9529ba4904c615e,0.000000,0.064375,0.069167,0.177500,Car
422,f9529ba4904c615e,0.029375,0.271875,0.079167,0.275833,Car
423,f9529ba4904c615e,0.041250,0.096250,0.066667,0.100000,Car
424,f9529ba4904c615e,0.176875,0.238750,0.063333,0.127500,Car


Use the exportnames.py file to generate image.txt (list of images for sample of 100 images)

In [13]:
img_dir = os.path.join(BASE_DIR, "data_pipeline/flow/images")
label_file_path = os.path.join(BASE_DIR, "data_pipeline/flow/image.txt") 

with open(label_file_path, 'r') as f:
    label_entries = [line.strip() for line in f.readlines()]
label_names = {entry.split('/')[-1] for entry in label_entries}
img_files = glob.glob(os.path.join(img_dir, "*.jpg"))
img_names = {os.path.splitext(os.path.basename(f))[0] for f in img_files}
missing_labels = img_names - label_names
missing_images = label_names - img_names

print(f"Num images: {len(img_files)}")
print(f"Num labels: {len(label_names)}")
print(f"Images with missing labels: {len(missing_labels)}")
print(f"Labels without image: {len(missing_images)}")

if missing_labels:
    print("List image with missing labels:", missing_labels)
if missing_images:
    print("List labels without image:", missing_images)


Num images: 100
Num labels: 100
Images with missing labels: 0
Labels without image: 0


In [16]:
!mkdir dataset 

In [17]:
# Define paths
train_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/images/train")
val_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/images/val")
test_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/images/test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
img_files = glob.glob(os.path.join(img_dir, "*.jpg"))

random.seed(1610)
random.shuffle(img_files)

train_split = int(len(img_files) * 0.7)
val_split = int(len(img_files) * 0.85)

train_files = img_files[:train_split]
val_files = img_files[train_split:val_split]
test_files = img_files[val_split:]

for src_path in train_files:
    filename = os.path.basename(src_path)
    dst_path = os.path.join(train_dir, filename)
    shutil.copy2(src_path, dst_path)
    
for src_path in val_files:
    filename = os.path.basename(src_path)
    dst_path = os.path.join(val_dir, filename)
    shutil.copy2(src_path, dst_path)
    
for src_path in test_files:
    filename = os.path.basename(src_path)
    dst_path = os.path.join(test_dir, filename)
    shutil.copy2(src_path, dst_path)

print(f"Split complete: {len(train_files)} training images, {len(val_files)} validation images, and {len(test_files)} test images")

Split complete: 70 training images, 15 validation images, and 15 test images


In [22]:
labels = pd.read_csv(os.path.join(BASE_DIR, "data_pipeline/flow/labels.csv"))
labels['LabelName_Text'].unique()

array(['Car', 'Truck', 'Bus', 'Taxi'], dtype=object)

In [23]:
labels_train_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/labels/train")
labels_val_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/labels/val")
labels_test_dir = os.path.join(BASE_DIR, "tracking_pipeline/dataset/labels/test")

os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)
os.makedirs(labels_test_dir, exist_ok=True)

labels = pd.read_csv(os.path.join(BASE_DIR, "data_pipeline/flow/labels.csv"))

unique_labels = sorted(labels['LabelName_Text'].unique())
class_mapping = {label: idx for idx, label in enumerate(unique_labels)}

print("Class mapping:")
for label, class_id in class_mapping.items():
    print(f"  {label}: {class_id}")

train_images = {os.path.splitext(img)[0] for img in os.listdir(train_dir) if img.endswith(".jpg")}
val_images = {os.path.splitext(img)[0] for img in os.listdir(val_dir) if img.endswith(".jpg")}
test_images = {os.path.splitext(img)[0] for img in os.listdir(test_dir) if img.endswith(".jpg")}
for image_id, group in labels.groupby("ImageID"):
    if image_id in train_images:
        txt_path = os.path.join(labels_train_dir, f"{image_id}.txt")
    elif image_id in val_images:
        txt_path = os.path.join(labels_val_dir, f"{image_id}.txt")
    elif image_id in test_images:
        txt_path = os.path.join(labels_test_dir, f"{image_id}.txt")
    else:
        continue
        
    with open(txt_path, "w") as f:
        for _, row in group.iterrows():
            class_id = class_mapping.get(row["LabelName_Text"], -1)
            if class_id == -1:
                continue
            x_center = (row["XMin"] + row["XMax"]) / 2
            y_center = (row["YMin"] + row["YMax"]) / 2
            width = row["XMax"] - row["XMin"]
            height = row["YMax"] - row["YMin"]
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print(f"Created YOLO labels in {labels_train_dir}, {labels_val_dir}, {labels_test_dir}")

Class mapping:
  Bus: 0
  Car: 1
  Taxi: 2
  Truck: 3
Created YOLO labels in /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/train, /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/val, /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/test


In [25]:
!ls {train_dir} | wc -l
!ls {val_dir} | wc -l
!ls {test_dir} | wc -l

70
15
15


Inference


In [35]:
#Fix the data.yaml according to this
# train: /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/train
# val: /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/val
# names:
#   0: Bus
#   1: Car
#   2: Taxi
#   3: Truck
#   # 4: Cart
#   # 5: Golf cart

In [26]:
!touch data.yaml


In [27]:
!curl -L -o yolo11n.pt https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5482k  100 5482k    0     0  2234k      0  0:00:02  0:00:02 --:--:-- 8759k


In [28]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
results = model.predict(source="dataset/images/test", save=True, project="runs/detect")


image 1/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/05014ec6d0c7bd13.jpg: 384x640 2 persons, 6 cars, 428.7ms
image 2/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/0ba915af48183d77.jpg: 480x640 2 cars, 3 trucks, 166.7ms
image 3/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/0e73a8333380ba68.jpg: 480x640 9 cars, 81.0ms
image 4/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/171691e2cd00225c.jpg: 448x640 6 persons, 2 cars, 1 teddy bear, 85.8ms
image 5/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/33de2f9027d215e1.jpg: 448x640 4 persons, 2 trucks, 5 umbrellas, 66.0ms
image 6/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/465076b7186e4ab7.jpg: 640x640 1 person, 6 cars, 111.8ms
image 7/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/481d5a3cd567bb7a.jpg: 512x640 14 persons, 2 cars, 94.9ms
image 8/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/

To-do: Remove labels for people, only keep cars

In [29]:
import mlflow
with mlflow.start_run():
    mlflow.log_artifact(os.path.join(BASE_DIR, "data_pipeline/flow/image.txt"))
    mlflow.log_artifact(os.path.join(BASE_DIR, "data_pipeline/flow/labels.csv"))
    mlflow.log_artifact("dataset/labels")
    mlflow.log_artifact("data.yaml")
    mlflow.log_artifact("runs/detect")

In [30]:
!mlflow ui --port 5000

[2025-04-23 13:36:45 +0700] [59725] [INFO] Starting gunicorn 23.0.0
[2025-04-23 13:36:45 +0700] [59725] [INFO] Listening at: http://127.0.0.1:5000 (59725)
[2025-04-23 13:36:45 +0700] [59725] [INFO] Using worker: sync
[2025-04-23 13:36:45 +0700] [59726] [INFO] Booting worker with pid: 59726
[2025-04-23 13:36:45 +0700] [59727] [INFO] Booting worker with pid: 59727
[2025-04-23 13:36:45 +0700] [59728] [INFO] Booting worker with pid: 59728
[2025-04-23 13:36:45 +0700] [59729] [INFO] Booting worker with pid: 59729
[2025-04-23 13:38:31 +0700] [59725] [INFO] Handling signal: int
^C

Aborted!
[2025-04-23 13:38:32 +0700] [59728] [INFO] Worker exiting (pid: 59728)
[2025-04-23 13:38:32 +0700] [59729] [INFO] Worker exiting (pid: 59729)
[2025-04-23 13:38:32 +0700] [59727] [INFO] Worker exiting (pid: 59727)
[2025-04-23 13:38:32 +0700] [59726] [INFO] Worker exiting (pid: 59726)


In [31]:
import mlflow
with mlflow.start_run():
    mlflow.log_artifact(os.path.join(BASE_DIR, "data_pipeline/flow/image.txt"))
    mlflow.log_artifact(os.path.join(BASE_DIR, "data_pipeline/flow/labels.csv"))
    mlflow.log_artifact("dataset/labels")
    mlflow.log_artifact("data.yaml")
    mlflow.log_artifact("runs/detect")

Train 

In [33]:
import os
command = (
    "yolo train "
    "data=data.yaml "
    "model=yolo11n.pt "
    "epochs=20 "
    "batch=8 "
    "imgsz=416 "
    "device=cpu "
    "project=runs/train "
    "name=exp5"
)

print("Starting training...")
os.system(command)
print("Training finished!")

Starting training...
Ultralytics 8.3.114 🚀 Python-3.10.16 torch-2.6.0+cu124 CPU (AMD Ryzen 7 8845H w/ Radeon 780M Graphics)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs/train, name=exp52, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

100%|██████████| 755k/755k [00:00<00:00, 2.84MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/train... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<00:00, 450.73it/s]


train: New cache created: /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1888.4±637.2 MB/s, size: 213.1 KB)
val: New cache created: /home/meth/code/mlopsfin/tracking_pipeline/dataset/labels/val.cache


...

                   all         15         61      0.794      0.238      0.251      0.142

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20         0G      1.291      1.478      1.198         37        416: 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


                   all         15         61      0.823      0.233       0.25      0.143

20 epochs completed in 0.041 hours.
Optimizer stripped from runs/train/exp52/weights/last.pt, 5.4MB
Optimizer stripped from runs/train/exp52/weights/best.pt, 5.4MB

Validating runs/train/exp52/weights/best.pt...
Ultralytics 8.3.114 🚀 Python-3.10.16 torch-2.6.0+cu124 CPU (AMD Ryzen 7 8845H w/ Radeon 780M Graphics)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


                   all         15         61      0.822      0.233       0.25      0.143
                   Car         15         60      0.644      0.467      0.501      0.285
                  Taxi          1          1          1          0          0          0
Speed: 0.4ms preprocess, 25.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/train/exp52
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
💡 Learn more at https://docs.ultralytics.com/modes/train
Training finished!


Predict again with best model

In [34]:
from ultralytics import YOLO
model = YOLO("runs/train/exp52/weights/best.pt")
results = model.predict(source="dataset/images/test", save=True, project="runs/detect", name="predict_new")


image 1/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/05014ec6d0c7bd13.jpg: 256x416 6 Cars, 105.8ms
image 2/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/0ba915af48183d77.jpg: 320x416 1 Car, 85.3ms
image 3/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/0e73a8333380ba68.jpg: 320x416 3 Cars, 60.2ms
image 4/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/171691e2cd00225c.jpg: 288x416 2 Cars, 74.8ms
image 5/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/33de2f9027d215e1.jpg: 288x416 (no detections), 41.9ms
image 6/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/465076b7186e4ab7.jpg: 416x416 5 Cars, 112.0ms
image 7/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/481d5a3cd567bb7a.jpg: 352x416 1 Car, 102.5ms
image 8/15 /home/meth/code/mlopsfin/tracking_pipeline/dataset/images/test/5400848ead05f765.jpg: 256x416 3 Cars, 46.7ms
image 9/15 /home/meth/code/mlopsfin/t

In [36]:
!mlflow ui --port 5000

[2025-04-23 13:51:51 +0700] [61995] [INFO] Starting gunicorn 23.0.0
[2025-04-23 13:51:51 +0700] [61995] [INFO] Listening at: http://127.0.0.1:5000 (61995)
[2025-04-23 13:51:51 +0700] [61995] [INFO] Using worker: sync
[2025-04-23 13:51:51 +0700] [61996] [INFO] Booting worker with pid: 61996
[2025-04-23 13:51:51 +0700] [62000] [INFO] Booting worker with pid: 62000
[2025-04-23 13:51:51 +0700] [62001] [INFO] Booting worker with pid: 62001
[2025-04-23 13:51:51 +0700] [62005] [INFO] Booting worker with pid: 62005
[2025-04-23 13:53:41 +0700] [61995] [INFO] Handling signal: int
^C

Aborted!
[2025-04-23 13:53:41 +0700] [62001] [INFO] Worker exiting (pid: 62001)
[2025-04-23 13:53:41 +0700] [62005] [INFO] Worker exiting (pid: 62005)
[2025-04-23 13:53:41 +0700] [62000] [INFO] Worker exiting (pid: 62000)
[2025-04-23 13:53:41 +0700] [61996] [INFO] Worker exiting (pid: 61996)
